# 最尤法によるパラメータ推定

## 概要
カルマンフィルタの適用に必要となるパラメータの推定方法（最尤法）に関して

## 最尤法適用の流れ
1. 適当なパラメータでカルマンフィルターを実行し、状態を推定
1. この時の尤度（どれほど尤もらしくデータを表現できているか）を計算
1. 尤度を最大化するようにパラメータを更新
1. 尤度が最大化したパラメータを使って状態空間モデルを推定しなおす（尤もらしい状態空間モデルの出来上がり）

## 尤度の計算方法
ローカルレベルモデルでは、正規分布が仮定される → 平均・分散が尤度の計算に欲しい

- 尤度計算に用いるデータ: 観測値の予測誤差 = 観測値 - 観測値の予測結果
- 平均: 0 (誤差の期待値が0とするのは自然）
- 分散: 観測値の予測誤差の分散
    - (ある時点における観測値の予測誤差の分散) = (当該時点の状態の予測誤差の分散) + (観測方程式のノイズの分散)
    - なお、(当該時点の状態の予測誤差の分散) = (1時点前の状態の予測誤差の分散) + (状態方程式のノイズの分散)

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# グラフを横長にする
plt.rcParams['figure.figsize'] = 15, 6
# 文字サイズの指定
plt.rcParams["font.size"] = 18

### データの用意

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                                    index_col='Month', parse_dates=True,  dtype='float')
ts = data['Passengers']

### 線形ガウス状態空間モデルにローカルレベルモデルを想定

**状態方程式**: $x_t = x_{t-1} + \varepsilon_t$<br>
**観測方程式**: $y_t = x_{t} + \eta_t$

- 入力
    - y: 当該時点の観測値
    - x_pre: 1時点前の状態推定値
    - x_err_var_pre: 1時点前の状態の予測誤差の分散
    - sigma_y: 観測方程式の誤差項の分散 ($\eta_t$の分散)
    - sigma_x: 状態方程式の誤差項の分散 ($\varepsilon_t$の分散)
- 出力
    - x_current: 当該時点の状態の推定値（カルマンフィルタ適用後）
    - x_err_var_current: 当該時点の状態の予測誤差の分散（カルマンフィルタによる状態の補正を考慮）
    - y_err: 観測値の予測誤差
    - y_err_var: 観測値の予測誤差の分散

In [6]:
def local_level_model(y, x_pre, x_err_var_pre, sigma_y, sigma_x):
    # 今期の状態の予測　（状態方程式により算出）
    x_current = x_pre + np.random.normal(loc=0, scale=np.sqrt(sigma_x))
    
    # 今期の予測誤差の分散 = 前期の予測誤差の分散 + 状態方程式のノイズの分散
    x_err_var_current = x_err_var_pre + sigma_x
    
    # 今期の状態の推定値から観測値を予測
    y_pred = x_current + np.random.normal(loc=0, scale=np.sqrt(sigma_y))
    
    # 観測値の予測誤差
    y_err = y - y_pred
    
    # 観測値の予測誤差の分散 = 当該時点の状態の予測誤差の分散 + 観測方程式のノイズの分散
    y_err_var = x_err_var_current + sigma_y
    
    # カルマンゲイン: 状態をどんくらい補正するか
    k_gain = x_err_var_current / (x_err_var_current + sigma_y)
    
    # 状態の補正
    x_current += k_gain * y_err
    
    # 状態の予測誤差の分散の修正
    x_err_var_current *= (1 - k_gain)
    
    return x_current, x_err_var_current, y_err, y_err_var

In [12]:
def calc_loglikelihood(x_0, x_err_var_0, std_x, std_y):
    # パラメータ更新時に分散が負になることを回避
    sigma_x = std_x ** 2
    sigma_y = std_y ** 2
    
    # 状態, 初期値は x_0
    x = ts.copy()

    # 観測値の推定結果
    y_err = ts.copy()
    y_err_var = ts.copy()

    # 状態の予測誤差の分散, 初期値は x_err_var_0
    x_err_var = np.append(x_err_var_0, np.zeros(len(ts)))
    
    # カルマンフィルタによる状態の補正（推定）
    for i in range(len(ts)):
        if i == 0:
            x[i], x_err_var[i+1], y_err[i], y_err_var[i] = local_level_model(ts[i], x_0, x_err_var[i], sigma_y=sigma_y, sigma_x=sigma_x)
        else:
            x[i], x_err_var[i+1], y_err[i], y_err_var[i] = local_level_model(ts[i], x[i-1], x_err_var[i], sigma_y=sigma_y, sigma_x=sigma_x)

    #　対数尤度 = - N / 2 * np.log(2 * np.pi) - 1 / 2 * (np.log(sigma^2) + y_err^2 / sigma^2)
    # 定数部分を除去 + 負の対数尤度の最小化問題にする
    loglikelihood = sum(np.log(y_err_var) + y_err ** 2 / y_err_var) / 2
    return loglikelihood

In [22]:
calc_loglikelihood(x_0=1e2, x_err_var_0=1e2, std_x=1e2, std_y=1e2)

811.7659203197206